In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tool import *
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side
from PIL import ImageGrab
import win32com.client
import pyautogui

In [2]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)

In [3]:
def type_email_recipents(recipent_email, recipent_type):
    browser.switch_to.frame('Main') # 프레임 이동
    browser.switch_to.frame(browser.find_elements(By.TAG_NAME, "iframe")[1]) # frame 이동
    browser.switch_to.frame(browser.find_elements(By.TAG_NAME, "frame")[1]) # 한번더 frame 이동
    for sp in browser.find_elements_by_tag_name('span'): # 수신자를 참조로 선택
        if sp.text == recipent_type:
            elem = sp
            elem.click()
            break
    elem = browser.find_element_by_name('WebTCB') # 메일수신자 입력 선택
    elem.send_keys(recipent_email)
    elem.send_keys(Keys.ENTER)
    browser.switch_to.default_content()

def enter_subject(email_subject):
    browser.switch_to.frame('Main') # 프레임 이동
    browser.switch_to.frame(browser.find_elements(By.TAG_NAME, "iframe")[1]) # frame 이동
    browser.switch_to.frame(browser.find_elements(By.TAG_NAME, "frame")[1]) # 한번더 frame 이동
    elem = browser.find_element_by_name('WebSubject')
    elem.send_keys(email_subject)
    browser.switch_to.default_content()

def write_email(content):
    browser.switch_to.frame('Main') # 프레임 이동
    browser.switch_to.frame(browser.find_elements(By.TAG_NAME, "iframe")[1]) # frame 이동
    browser.switch_to.frame(browser.find_elements(By.TAG_NAME, "frame")[1]) # 한번더 frame 이동
    browser.switch_to.frame(browser.find_element_by_xpath('//*[@id="tx_canvas_wysiwyginstance_1"]')) # 메일 작성을 위한 iframe 으로 이동
    elem = browser.find_element_by_xpath('/html/body/p[1]') # 메일 본문 선택
    elem.send_keys(content)
    browser.switch_to.default_content()

def compose_email():
    browser.switch_to.frame('Main') # 프레임 이동
    browser.find_element_by_xpath('//*[@id="id_button_group_item_UI_INSTANCE_752"]/div/div[1]/div[2]').click() # 메일 작성 버튼 클릭 
    browser.switch_to.default_content()

def save_email():
    browser.switch_to.frame('Main') # 프레임 이동
    browser.switch_to.frame(browser.find_elements(By.TAG_NAME, "iframe")[1]) # frame 이동
    browser.switch_to.frame(browser.find_elements(By.TAG_NAME, "frame")[1]) # 한번더 frame 이동
    browser.find_element_by_xpath('//*[@id="id_button_group_item_UI_INSTANCE_7"]/div/div[1]/div[2]').click() # 메일 본문 저장

def copy_excel_table(filepath):
    excel = win32com.client.Dispatch('Excel.Application')
    excel.visible = True
    wb = excel.Workbooks.Open(filename)
    ws = wb.ActiveSheet
    ws.UsedRange.CopyPicture(Format=2) # 엑셀 영역을 image로 변환하여 복사
    ImageGrab.grabclipboard() # 클립보드의 이미지를 읽어옴
    write_email((Keys.CONTROL, 'v'))
    wb.Save()
    excel.quit()

def edit_excel(filename, confirm_wk):
    wb = load_workbook(filename)
    color = PatternFill(fgColor='9999FF', fill_type='solid') # violet
    confirm_color = PatternFill(fgColor='00cc00', fill_type='solid') # Honeydew
    pseudo_confirm_color = PatternFill(fgColor='9ACD32', fill_type='solid') # Yellowgreen
    border_style = Side(border_style="thin", color="000000")
    border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)
    sh = wb[wb.sheetnames[0]]
    sh['E1'] = 'Sub TTL'
    for i in range(6, sh.max_column+1):
        col_letter = get_column_letter(i)
        sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}7:{col_letter}{sh.max_row})'
        sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
    for row in sh.iter_rows(min_row=2, min_col=5, max_row=5, max_col=sh.max_column):
        for cell in row:
            cell.fill = color
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    merge_range = f'{get_column_letter(sh.max_column)}2:{get_column_letter(sh.max_column)}5'
    sh.merge_cells(merge_range)
    for cell in sh[merge_range][0]:
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    for row in sh.iter_rows(min_row=6, min_col=1, max_row=6, max_col=5):
        for cell in row:
            cell.fill = color
    for row in sh.iter_rows(min_row=7, min_col=6, max_row=sh.max_row, max_col=sh.max_column):
        for cell in row:
            cell.border = border
            cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
    for row in sh.iter_rows(min_row=7, min_col=5):
        if row[0].value == confirm_wk:
            for cell in row:
                cell.fill = confirm_color
        elif row[0].value in [get_weekname_from(confirm_wk, i) for i in range(1, 4)]:
            for cell in row:
                cell.fill = pseudo_confirm_color
    sh.freeze_panes = 'F7'
    sh.column_dimensions['A'].width = 6
    sh.column_dimensions['B'].width = 6
    sh.column_dimensions['E'].width = 17.5
    sh.sheet_view.showGridLines = False
    for i in range(6, sh.max_column+1):
        sh.column_dimensions[get_column_letter(i)].width = 6.5
    wb.save(filename)
    wb.close()

# 1. 메일 작성을 위한 데이터 사전 준비

In [4]:
ver_table = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index()
ver_table = ver_table[ver_table['From Site'].str.contains('TCL')]
ver_table.iloc[-30:, ]

,Planweek,From Site,Category,Created_at,SP
435,2023-11-27(W48),OS_TCL_CN_P,Latest26w,202311270912,430189.0
436,2023-11-27(W48),OS_TCL_CN_P,Latest26w,202311271526,440436.0
437,2023-11-27(W48),OS_TCL_CN_P,Latest26w,202311280839,431764.0
466,2023-12-04(W49),OS_TCL_CN_P,Final26w,202312060916,404149.0
467,2023-12-04(W49),OS_TCL_CN_P,Final52w,202312051437,586409.0
468,2023-12-04(W49),OS_TCL_CN_P,Latest26w,202312040926,397415.0
469,2023-12-04(W49),OS_TCL_CN_P,Latest26w,202312041556,403883.0
470,2023-12-04(W49),OS_TCL_CN_P,Latest26w,202312050956,404187.0
471,2023-12-04(W49),OS_TCL_CN_P,Latest26w,202312070937,404149.0
502,2023-12-11(W50),OS_TCL_CN_P,Final26w,202312130843,389916.0


In [5]:
i1 = 601
i2 = 574

p_wk = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['Planweek']
vendor = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['From Site'].split('_')[1]
ver = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['Category']

sp1 = spdb.merge(spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i1], :4])
sp2 = spdb.merge(spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i2], :4])
sp2 = sp2[sp2['Week Name'] >= sp1['Week Name'].min()]

sp = pd.concat([sp1, sp2])
sp['Month'] = sp['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')

In [6]:
sp = sp[sp['Week Name'] >= p_wk]
sp = sp[sp['Month'] <= '2024-04']

In [7]:
sp.insert(7, 'Series', sp['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0]))
sp.insert(7, 'Inch', sp['Mapping Model.Suffix'].apply(lambda x:x[:2]))
sp = pd.merge(sp, site[['To Site', 'Subsidiary', 'Country']], how='left')
sp.loc[sp['Division']=='PC', 'SCM'] = sp.loc[sp['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['Division']=='PC', 'MKT'] = sp.loc[sp['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['Division']=='MNT', 'SCM'] = sp.loc[sp['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['Division']=='MNT', 'MKT'] = sp.loc[sp['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
us_b2b = sp.loc[sp['To Site'].str.contains('ENUS') & (sp['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].min())
sp.loc[sp['To Site'].str.contains('ENUS') & (sp['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2b
sp = sp.fillna('N/A')

In [8]:
gap = sp.groupby(['MKT', 'SCM', 'Planweek', 'Created_at', 'To Site', 'Category', 'Series', 'Month', 'Week Name'])['SP'].sum().unstack(['Planweek', 'Category', 'Created_at']).fillna(0).astype(int)
gap['Gap'] = gap[gap.columns[-1]] - gap[gap.columns[0]]
gap = gap.unstack('Month').fillna(0).astype(int)
gap['Sum of Gap'] = gap['Gap'].sum(axis=1)
gap = gap.reset_index()
명단 = gap['MKT'].unique()

In [9]:
# 엑셀만 만들 때
for 담당자 in 명단:
    df = gap[gap['MKT'] == 담당자].copy()
    site_name = ', '.join(df['To Site'].unique())
    updated_time = datetime.datetime.now().strftime('%Y%m%d%H%M')
    filename = f'D:/SP검토/{updated_time}_{담당자}_{p_wk}_TCL SP 검토_{site_name}.xlsx'
    df.reset_index(drop=True).set_index(['MKT', 'SCM', 'To Site', 'Series', 'Week Name']).to_excel(filename, startrow=1)
    edit_excel(filename, get_weekname_from(p_wk, 4))    

# 2. 크롬 띄우고 EP 메일 창 활성화

In [32]:
browser = webdriver.Chrome()
browser.maximize_window()
browser.get('https://sso.lge.com/') # EP 접속

# EP 로그인 후 10초 대기(지문인식 시간)
elem = browser.find_element_by_id('USER') 
elem.send_keys('paul76.lee')
elem.send_keys(Keys.ENTER)
browser.find_element_by_id('loginBiobtn').click()
pyautogui.sleep(15)

elem = browser.find_element_by_class_name('quick_big_count')
elem.click() # EP 메일 클릭
pyautogui.sleep(10)
browser.switch_to.window(window_name=browser.window_handles[1]) # EP mail 탭 전환

# 3. 메일 작성

In [33]:
for 담당자 in 명단:
    df = gap[gap['MKT'] == 담당자].copy()
    site_name = ', '.join(df['To Site'].unique())
    updated_time = datetime.datetime.now().strftime('%Y%m%d%H%M')
    filename = f'D:/SP검토/{updated_time}_{담당자}_{p_wk}_TCL SP 검토_{site_name}.xlsx'
    df.reset_index(drop=True).set_index(['MKT', 'SCM', 'To Site', 'Series', 'Week Name']).to_excel(filename, startrow=1)
    edit_excel(filename, get_weekname_from(p_wk, 4))    
    직급 = cont[cont['Name'] == 담당자]['Position'].values[0]
    email_address = cont.loc[cont['Name'] == 담당자, 'Email'].values[0]
    c1 = cont['Subsidiary']== cont.loc[cont['Name'] == 담당자, 'Subsidiary'].values[0]
    c2 = cont['Div'] == 'MNT'
    c3 = cont['Team'] == 'SCM'
    cc_address = cont[c1 & c2 & c3]['Email'].values[0]
    subject = f"{p_wk} TCL 모니터 {site_name}향 Main SP 검토 요청"
    메일본문 = f'''
    안녕하세요. 

    수신 : {담당자} {직급}님,

    수고 많으십니다.
    첨부는 금일 오후 3시 기준의 전주 대비 TCL 모니터의 {p_wk} 기준 Main SP 현황입니다.
    금주 {get_weekname_from(p_wk, 4)} 확정주차이며 하기에 반영된 SP로 검토하겠습니다.
    
    몇주간 공지드린바와 같이 금주 확정 시, 아래 색깔로 표시된 춘절 직전 주차인 2/5W 까지 가확정할 예정이오니 현재 나온 SP의 적확도에 대해 면밀히
    검토를 부탁 드립니다. 차주에는 2/19W 까지 가확정 할 예정입니다.
    
    
    '''
    공지사항 = '''
    ※ 중요 공지사항 : 몇 주간 공지 드린바와 같이 24년 2월은 중국 춘절 영향으로 춘절 연휴인 2/12W는 SP 반영이 불가하며 
    춘절 연휴로 인한 작업자 복귀 문제, 자재 공급 업체의 자재 사전 Booking이 필요하여 현재 반영된 1~2월 전체 수량을 면밀히
    점검하여 사전에 확정이 필요합니다. 
    차주(12/25W) 에는 2/5W 까지 가확정할 예정이며, 
    차차주(1/1W) 에는 2/19W 까지 가확정이 필요하오니,     
    이 때 반영된 SP 이후에 증량 요청하실 경우 대응이 불가하며, 감량할 경우에 set 재고로 남게되오니
    각 법인별로 더 이상 변동이 없을 수량으로 반영을 부탁 드립니다.
    '''
    
    compose_email()
    type_email_recipents(email_address, 'To')
    type_email_recipents(cc_address, 'Cc')
    type_email_recipents('paul76.lee@lge.com', 'Bcc')
    enter_subject(subject)
    
    write_email(메일본문)
    
    copy_excel_table(filename)
    
    write_email(공지사항)
    
    save_email()
    pyautogui.sleep(2)
    browser.refresh()
    pyautogui.sleep(5)

In [34]:
browser.quit()

In [16]:
for 담당자 in 명단:
    df = gap[gap['MKT'] == 담당자].copy()
    site_name = ', '.join(df['To Site'].unique())
    updated_time = datetime.datetime.now().strftime('%Y%m%d%H%M')
    filename = f'D:/SP검토/{updated_time}_{담당자}_{p_wk}_TCL SP 검토_{site_name}.xlsx'
    df.reset_index(drop=True).set_index(['MKT', 'SCM', 'To Site', 'Series', 'Week Name']).to_excel(filename, startrow=1)
    edit_excel(filename, get_weekname_from(p_wk, 4))
    직급 = cont[cont['Name'] == 담당자]['Position'].values[0]
    email_address = cont.loc[cont['Name'] == 담당자, 'Email'].values[0]
    subject = f"{p_wk} TCL 모니터 {site_name}향 Main SP 검토 요청"
    메일본문 = f'''
    안녕하세요. 

    수신 : {담당자} {직급}님,

    수고 많으십니다.
    첨부는 전주 대비 TCL 모니터의 {p_wk} 기준 SP 현황입니다.
    금주 {get_weekname_from(p_wk, 4)} 확정주차이며 하기에 반영된 SP로 검토하겠습니다.
    
    감사합니다.
    이종목 드림
    '''
    
    print(subject)
    print(메일본문)
#     print(공지사항)

2023-12-04(W49) TCL 모니터 EAIL향 Main SP 검토 요청

    안녕하세요. 

    수신 : 고주현 책임님,

    수고 많으십니다.
    첨부는 전주 대비 TCL 모니터의 2023-12-04(W49) 기준 SP 현황입니다.
    금주 2024-01-01(W01) 확정주차이며 하기에 반영된 SP로 검토하겠습니다.
    
    감사합니다.
    이종목 드림
    
2023-12-04(W49) TCL 모니터 EMGF, EMGF_ECHODUBAI_DO, EMGF_MES_DO, EMGF_NAAM_DO, EMGF_PCI_DO, EMGF_TRIGON_DO향 Main SP 검토 요청

    안녕하세요. 

    수신 : 김대성 책임님,

    수고 많으십니다.
    첨부는 전주 대비 TCL 모니터의 2023-12-04(W49) 기준 SP 현황입니다.
    금주 2024-01-01(W01) 확정주차이며 하기에 반영된 SP로 검토하겠습니다.
    
    감사합니다.
    이종목 드림
    
2023-12-04(W49) TCL 모니터 EEES향 Main SP 검토 요청

    안녕하세요. 

    수신 : 김원중 책임님,

    수고 많으십니다.
    첨부는 전주 대비 TCL 모니터의 2023-12-04(W49) 기준 SP 현황입니다.
    금주 2024-01-01(W01) 확정주차이며 하기에 반영된 SP로 검토하겠습니다.
    
    감사합니다.
    이종목 드림
    
2023-12-04(W49) TCL 모니터 EJJP향 Main SP 검토 요청

    안녕하세요. 

    수신 : 김재영 사원님,

    수고 많으십니다.
    첨부는 전주 대비 TCL 모니터의 2023-12-04(W49) 기준 SP 현황입니다.
    금주 2024-01-01(W01) 확정주차이며 하기에 반영된 SP로 검토하겠습니다.
    
    감사합니다.
    이종목 드림
    
2023-12-04(W49) TCL 모